# Marine eutrophication impact factors 

Exiobase categories:
- N - agriculture - water

LC-Impact stressors
- Marine eutrophication

In [4]:
# import required libraries
import pymrio
import numpy as np
import pandas as pd
import pycountry as pyc
import json

# load arguments from json file
with open("../arguments.json", "r") as f:
    arguments = json.load(f)

In [5]:
# exiobase 2011 is used for calculating share of stressor for each region-product pair
exio3_11 = pymrio.parse_exiobase3(path=arguments["exio_11_path"])
# exiobase 2019 is used for impact factors
exio3_19 = pymrio.parse_exiobase3(path=arguments["exio_19_path"])

## Calculate DRorigin
DRorigin is a matrix which describes the amount of the driver of biodiversity loss (DR) that occurs in impact region i sector k and is driven by consumption in region j sector k.

To calculate the matrix we need to
1. Aggregate relevant drivers from exiobase together
2. Diagonalize the aggregated driver and re-calculate the IO system

In [6]:
# no need for aggregation

In [7]:
# diagonalize stressor
# optimized for speed
diag_water = exio3_11.satellite.diag_stressor(("N - agriculture - water"))

# First, make sure that L matrix is calculated 
if exio3_11.L is None:
    # try loading the L matrix from pickles/exio3_11_L.pickle
    try:
        exio3_11.L = pd.read_pickle("pickles/exio3_11_L.pickle")
        print("L matrix loaded from pickle")
    except FileNotFoundError:
        print("L matrix not found, calculating it from scratch.")
        print("Calculating A")
        exio3_11.A = pymrio.calc_A(exio3_11.Z, exio3_11.x)
        print("Calculating L")
        exio3_11.L = pymrio.calc_L(exio3_11.A)
        # save the L matrix to a pickle file
        pd.to_pickle(exio3_11.L, "pickles/exio3_11_L.pickle")
else:
    print("L already loaded")

# calculate S (direct emission multipliers)
if diag_water.S is None:
    print("Calculating S")
    diag_water.S = pymrio.calc_S(diag_water.F, exio3_11.x)

Y_agg = exio3_11.Y.groupby(level="region", axis=1, sort=False).sum()

diag_water.D_cba, _, _, _ = pymrio.calc_accounts(diag_water.S, exio3_11.L, Y_agg)
diag_water.D_cba

L matrix loaded from pickle
Calculating S


/var/folders/bg/8zzl9cj57fnd569zy539k5h40000gn/T/ipykernel_59579/39903974.py:27: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = exio3_11.Y.groupby(level="region", axis=1, sort=False).sum()


region                                                              AT  \
sector                                                      Paddy rice   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              6.402791e-03   
       Cereal grains nec                                  2.142313e-02   
       Vegetables, fruit, nuts                            2.720965e-04   
       Oil seeds                                          2.384799e-03   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       6.451847e-05   
       Recreational, cultural and sporting services (92)  1.882675e-03   
       Other services (93)                                1.580414e-04   
       Private households with employed persons (95)      5.587061e-07   
       Extra-territorial organizations and bodies         0.000000e+00   

region                                                                   \
sector                                                            Wheat   
region sector                                                             
AT     Paddy rice                                              0.000000   
       Wheat                                              608974.564598   
       Cereal grains nec                                      23.602639   
       Vegetables, fruit, nuts                                 1.168402   
       Oil seeds                                               5.970210   
...                                                                 ...   
WM     Membership organisation services n.e.c. (91)            0.008239   
       Recreational, cultural and sporting services (92)       0.286583   
       Other services (93)                                     0.016363   
       Private households with employed persons (95)           0.000088   
       Extra-territorial organizations and bodies              0.000000   

region                                                                      \
sector                                                   Cereal grains nec   
region sector                                                                
AT     Paddy rice                                             0.000000e+00   
       Wheat                                                  8.360758e+01   
       Cereal grains nec                                      4.098168e+06   
       Vegetables, fruit, nuts                                6.849809e+00   
       Oil seeds                                              2.958343e+01   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)           2.722233e-02   
       Recreational, cultural and sporting services (92)      1.078214e+00   
       Other services (93)                                    5.557015e-02   
       Private households with employed persons (95)          3.109801e-04   
       Extra-territorial organizations and bodies             0.000000e+00   

region                                                                            \
sector                                                   Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                   0.000000e+00   
       Wheat                                                        7.244308e+02   
       Cereal grains nec                                            1.356209e+03   
       Vegetables, fruit, nuts                                      3.079774e+06   
       Oil seeds                                                    2.068045e+02   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)            

## Calculate DR share
DR share is a new matrix that represents the share of the driver in the impact region i from the total amount of driver that is driven by consumption in region j sector k.

To calculate the matrix each column of DR origin is shared by the sum of that column.

In [8]:
# calculate dr_s - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k
columns = {}
for series_name, series in diag_water.D_cba.items():
    series_sum = series.sum()
    columns[series_name] = series / series_sum

dr_s_w = pd.DataFrame(columns)
dr_s_w

AT  \
                                                            Paddy rice   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              3.278069e-07   
       Cereal grains nec                                  1.096811e-06   
       Vegetables, fruit, nuts                            1.393066e-08   
       Oil seeds                                          1.220957e-07   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       3.303184e-09   
       Recreational, cultural and sporting services (92)  9.638824e-08   
       Other services (93)                                8.091322e-09   
       Private households with employed persons (95)      2.860435e-11   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                                 Wheat   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              2.588195e-01   
       Cereal grains nec                                  1.003133e-05   
       Vegetables, fruit, nuts                            4.965812e-07   
       Oil seeds                                          2.537391e-06   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       3.501850e-09   
       Recreational, cultural and sporting services (92)  1.218002e-07   
       Other services (93)                                6.954595e-09   
       Private households with employed persons (95)      3.727299e-11   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                            \
                                                         Cereal grains nec   
region sector                                                                
AT     Paddy rice                                             0.000000e+00   
       Wheat                                                  1.010947e-05   
       Cereal grains nec                                      4.955330e-01   
       Vegetables, fruit, nuts                                8.282498e-07   
       Oil seeds                                              3.577103e-06   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)           3.291608e-09   
       Recreational, cultural and sporting services (92)      1.303731e-07   
       Other services (93)                                    6.719306e-09   
       Private households with employed persons (95)          3.760240e-11   
       Extra-territorial organizations and bodies             0.000000e+00   

                                                                                  \
                                                         Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                   0.000000e+00   
       Wheat                                                        7.846721e-05   
       Cereal grains nec                                            1.468987e-04   
       Vegetables, fruit, nuts                                      3.335877e-01   
       Oil seeds                                                    2.240017e-05   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                 1.327250e-07   
       Recreational, cultural and sporting services (92)     

## Calculate DR unit
DR unit is a region harmonized version of DR share.

To calculate DR unit we need to:
1. Identify regions that are missing from exiobase (rest of the world regions) but are present in lc-impact.
2. Assume that the impacts are divided evenly across the rest of the world category
3. By that assumption we can add the missing countries to DR share as the impact of the row region where country belongs to divided by the number of countries in that region

In [9]:
# harmonize regions 

# Load region mappings from arguments
row_eu_countries = arguments["row_region_mappings"]["row_eu"]
row_asia_pacific_countries = arguments["row_region_mappings"]["row_asia_pacific"]
row_african_countries = arguments["row_region_mappings"]["row_africa"]
row_american_countries = arguments["row_region_mappings"]["row_america"]
row_middle_eastern_countries = arguments["row_region_mappings"]["row_middle_east"]

exio_regions = exio3_11.get_regions()
row_regions = {
    "WA": "Asia and pacific",
    "WE": "Europe",
    "WF": "Africa",
    "WM": "Middle east",
    "WL": "America"
}

In [10]:
# Function to convert country name to ISO Alpha-2 code
def get_country_code(name):
    # custom mappings for countries that pycountry does not recognize
    # these should cover all the countries in the LCI data if country has alpha-2 code
    # these mappings were extracted manually
    extra_mappings = {
        "Turkey": "TR",
        "Russia": "RU",
        "Bahamas, The": "BS",
        "Bonaire": "BQ",
        "Byelarus": "BY",
        "Brunei": "BN",
        "Cape Verde": "CV",
        "Cocos Islands": "CC",
        "Congo DRC": "CD",
        "China, Hong Kong Special Administrative Region": "HK",
        "Curacao": "CW",
        "Democratic Republic of the Congo": "CD",
        "Falkland Islands": "FK",
        "Falkland Islands (Islas Malvinas)": "FK",
        "Gambia, The": "GM",
        "Gaza Strip": "PS",
        "Heard Island & McDonald Islands": "HM",
        "Ivory Coast": "CI",
        "Macedonia": "MK",
        "The Former Yugoslav Republic of Macedonia": "MK",
        "Macau": "MO",
        "Man, Isle of": "IM",
        "Micronesia": "FM",
        "Myanmar (Burma)": "MM",
        "Netherlands Antilles": "AN",
        "Palestinian Territory": "PS",
        "Pacific Islands (Palau)": "PW",
        "Pitcairn Islands": "PN",
        "Reunion": "RE",
        "Saba": "BQ",
        "Saint Eustatius": "BQ",
        "Saint Helena": "SH",
        "Saint Martin": "MF",
        "Sint Maarten": "SX",
        "South Georgia and the South Sandwich Is": "GS",
        "South Georgia": "GS",
        "St. Helena": "SH",
        "Saint Barthelemy": "BL",
        "Saint Kitts and Nevis": "KN",
        "St. Kitts and Nevis": "KN",
        "St. Lucia": "LC",
        "St. Pierre and Miquelon": "PM",
        "Sao Tomo and Principe": "ST",
        "St. Vincent and the Grenadines": "VC",
        "Svalbard": "SJ",
        "Jan Mayen": "SJ",
        "Swaziland": "SZ",
        "US Virgin Islands": "VI",
        "Virgin Islands": "VG",
        "Western Samoa": "WS",
        "West Bank": "PS",
    }
    try:
        return pyc.countries.lookup(name).alpha_2
    except LookupError:
        try:
            return extra_mappings[name]
        except LookupError:
            print("Alpha-2 country code does not exist for ", name)
            return None

In [11]:
# load and prepare lc-impact data
lci = pd.read_excel(arguments["lc_impact_path"] + "/9-marine eutrophication/CFs_marine_eutrophication.xlsx", sheet_name="country CFs")
lci = lci[["Country.1", "CF for direct N emission to marine system [PDF*yr/kg]"]]
lci.rename(columns={lci.columns[0]: "Country"}, inplace=True)

# Add country codes to all LCI datasets and remove missing ones
lci["Country_Code"] = lci["Country"].apply(get_country_code)

# Drop countries without alpha-2 code
lci.dropna(subset=["Country_Code"], inplace=True)
lci

Alpha-2 country code does not exist for  Baker Island
Alpha-2 country code does not exist for  French Southern & Antarctic Lands
Alpha-2 country code does not exist for  Glorioso Islands
Alpha-2 country code does not exist for  Howland Island
Alpha-2 country code does not exist for  Jarvis Island
Alpha-2 country code does not exist for  Johnston Atoll
Alpha-2 country code does not exist for  Juan De Nova Island
Alpha-2 country code does not exist for  Midway Islands
Alpha-2 country code does not exist for  Paracel Islands
Alpha-2 country code does not exist for  Spratly Islands
Alpha-2 country code does not exist for  Wake Island
Alpha-2 country code does not exist for  Zaire


,Country,CF for direct N emission to marine system [PDF*yr/kg],Country_Code
0,Afghanistan,0.000000e+00,AF
1,Albania,4.939992e-15,AL
2,Algeria,4.939992e-15,DZ
3,American Samoa,0.000000e+00,AS
4,Andorra,0.000000e+00,AD
...,...,...,...
235,Wallis and Futuna,0.000000e+00,WF
236,Western Sahara,1.288727e-15,EH
237,Western Samoa,0.000000e+00,WS
238,Yemen,2.680622e-15,YE


In [13]:
# add regional averages for regions that are not in LCI data

def get_missing_from_lci(exio_regions, lci):
    """
    Get the regions that are in exiobase but not in lci data.
    """
    missing = []
    for region in exio_regions:
        if region not in lci["Country_Code"].tolist():
            missing.append(region)
    return missing

def augment_marine(lci):
    n_w_eu = 3.86913531769129E-15
    luxembourg_row = pd.DataFrame({
        "Country": ["Luxembourg"],
        "Country_Code": ["LU"],
        "CF for direct N emission to marine system [PDF*yr/kg]": [n_w_eu],
    })
    switzerland_row = pd.DataFrame({
        "Country": ["Switzerland"],
        "Country_Code": ["CH"],
        "CF for direct N emission to marine system [PDF*yr/kg]": [n_w_eu],
    })
    lci = pd.concat([lci, luxembourg_row, switzerland_row], ignore_index=True)
    return lci

exio_regions_without_row = [region for region in exio_regions if region not in row_regions.keys()]
if len(get_missing_from_lci(exio_regions_without_row, lci)) > 0:
    print("Missing from LCI marine eutrophication:", get_missing_from_lci(exio_regions_without_row, lci))
    lci = augment_marine(lci)
    assert len(get_missing_from_lci(exio_regions_without_row, lci)) == 0, "There are still missing regions in marine eutrophication after augmentation"

Missing from LCI marine eutrophication: ['LU', 'CH']


In [15]:
# harmonize regions in LCI data

def get_row_regions(lci_country_codes, exio_country_codes):
    """
    Get the country codes from lci countries that don't exist in exiobase i.e. rest of the world countries.
    """
    row_regions = []
    for country in lci_country_codes:
        if country not in exio_country_codes:
            row_regions.append(country)
    
    # find duplicates in the list
    duplicates = []
    unique_regions = []
    seen_once = set()
    for item in row_regions:
        if item not in seen_once:
            unique_regions.append(item)
            seen_once.add(item)
        else:
            duplicates.append(item)
    if duplicates:
        print("Duplicates found in row regions:", duplicates)
    return unique_regions

exio_regions = exio3_11.get_regions()
row_countries = get_row_regions(lci["Country_Code"].tolist(), exio_regions)
print("Row regions for marine eutrophication:", row_countries)

# Load region mappings from arguments
row_eu_countries = arguments["row_region_mappings"]["row_eu"]
row_asia_pacific_countries = arguments["row_region_mappings"]["row_asia_pacific"]
row_african_countries = arguments["row_region_mappings"]["row_africa"]
row_american_countries = arguments["row_region_mappings"]["row_america"]
row_middle_eastern_countries = arguments["row_region_mappings"]["row_middle_east"]

Duplicates found in row regions: ['SJ', 'VG']
Row regions for marine eutrophication: ['AF', 'AL', 'DZ', 'AS', 'AD', 'AO', 'AI', 'AQ', 'AG', 'AR', 'AM', 'AW', 'AZ', 'BS', 'BH', 'BD', 'BB', 'BZ', 'BJ', 'BM', 'BT', 'BO', 'BA', 'BW', 'BV', 'IO', 'VG', 'BN', 'BF', 'BI', 'BY', 'KH', 'CM', 'CV', 'KY', 'CF', 'TD', 'CL', 'CX', 'CC', 'CO', 'KM', 'CG', 'CK', 'CR', 'CU', 'DJ', 'DM', 'DO', 'EC', 'EG', 'SV', 'GQ', 'ER', 'ET', 'FK', 'FO', 'FM', 'FJ', 'GF', 'PF', 'GA', 'GM', 'PS', 'GE', 'GH', 'GI', 'GL', 'GD', 'GP', 'GU', 'GT', 'GG', 'GN', 'GW', 'GY', 'HT', 'HM', 'HN', 'IS', 'IR', 'IQ', 'IL', 'CI', 'JM', 'SJ', 'JE', 'JO', 'KZ', 'KE', 'KI', 'KW', 'KG', 'LA', 'LB', 'LS', 'LR', 'LY', 'LI', 'MO', 'MK', 'MG', 'MW', 'MY', 'MV', 'ML', 'IM', 'MH', 'MQ', 'MR', 'MU', 'YT', 'MD', 'MC', 'MN', 'ME', 'MS', 'MA', 'MZ', 'MM', 'NA', 'NR', 'AN', 'NC', 'NZ', 'NI', 'NE', 'NG', 'NU', 'NF', 'KP', 'MP', 'OM', 'PW', 'PK', 'PA', 'PG', 'PE', 'PH', 'PN', 'PR', 'QA', 'RE', 'RW', 'ST', 'SA', 'SN', 'SC', 'SL', 'SG', 'SB', 'SO', 'G

In [16]:
# augment dr_s_w to create dr_u_w
# new regions are calculated by dividing their corresponding row region by the number of countries in the row region
# for example, row region Argentina is sub-matrix WA divided by the number of countries in row region WA
wl = dr_s_w.loc["WL"].copy()
wl = wl / len(row_american_countries)

we = dr_s_w.loc["WE"].copy()
we = we / len(row_eu_countries)

wa = dr_s_w.loc["WA"].copy()
wa = wa / len(row_asia_pacific_countries)

wf = dr_s_w.loc["WF"].copy()
wf = wf / len(row_african_countries)

wm = dr_s_w.loc["WM"].copy()
wm = wm / len(row_middle_eastern_countries)

dr_u_w = dr_s_w.copy()
dr_u_w = dr_u_w.drop(index=row_regions.keys(), level='region')

# build a mapping of country codes to region dataframes
country_to_region = {}
for region in row_countries:
    if region in row_eu_countries:
        country_to_region[region] = we
    elif region in row_asia_pacific_countries:
        country_to_region[region] = wa
    elif region in row_african_countries:
        country_to_region[region] = wf
    elif region in row_american_countries:
        country_to_region[region] = wl
    elif region in row_middle_eastern_countries:
        country_to_region[region] = wm
    else:
        raise ValueError(f"Unknown region: {region}")
    
# add all new regions that don't exist in exiobase to dr_u
all_indices = []
all_data = []
for region in row_countries:
    if region in country_to_region:  # Only add if we found a mapping
        region_data = country_to_region[region].copy()
        idx = pd.MultiIndex.from_product([[region],region_data.index], names=['region', 'sector'])
        all_indices.append(idx)
        all_data.append(region_data)
    else:
        print(f"ERROR: Region %s not found in country to region map, this should not happen", region)

combined_idx = pd.MultiIndex.from_tuples(
    [idx for subidx in all_indices for idx in subidx]
)

combined_data = pd.concat(all_data)
combined_data.index = combined_idx

dr_u_w = pd.concat([dr_u_w, combined_data])
# remove row regions from consumption regions (columns)
dr_u_w = dr_u_w.drop(columns=row_regions.keys(), axis=1, level=0)

## Calculate DR factor
Calculate the impact factors of the driver for each impact region i driven by consumption in region j product sector k.

These impact factors tell how

To calculate DR factor:
1. Calculate the monetary impact factor impact/€ from 2019 exiobase data
2. Multiply each column of DR unit with the impact factors of consumption region j in product sector k. The resulting matrix represents the distribution of impacts of 1€ consumption in each consumption region.

In [ ]:
# use 2019 impact factors for calculating dr_f
# calculate dr_f - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k

dr_f_w = dr_u_w.copy()
satellite_cleaned = exio3_19.satellite.M.drop(columns=row_regions.keys(), axis=1, level=0)
total = satellite_cleaned.loc["N - agriculture - water"]
scalars_w = total.to_numpy() # multipliers for each column
print(len(scalars_w))
print(dr_u_w.shape)

# multiply each column of dr_u_w by the respective column value from exio3_19.satellite_agg.M
dr_f_w = dr_f_w * scalars_w # same as dr_f_w * diag(scalars) but more efficient with numpy broadcasting
dr_f_w

9800
(45600, 8800)


ValueError: Unable to coerce to Series, length must be 8800: given 9800

## Calculate BDe factors (PDF/€ for each consumption region and product sector)

1. Multiply each cell of dr_u with the CF (characterisation factor from lc-impact) of the impact region to get PDF values for every entry
2. Sum up columns to get the total PDF/€ for the consumption region j product sector k

In [ ]:
# sort rows on lci in same order as dr_f.index.sortlevel
lci = lci.reindex(dr_f_w.index.get_level_values(0).unique())

# using average of P emissions to water and soil
# is this correct?
cf = lci["CF for direct N emission to marine system [PDF*yr/kg]"].to_numpy()
cf = np.repeat(cf, 200)
# expand cf_all_effects to match the shape of dr_f
cf = np.tile(cf, (dr_f_w.shape[1], 1)).T
print(cf.shape) # (37000, 9800)

# multiply each column of dr_f with cf_all_effects
pdf_w = dr_f_w * cf
pdf_w

(45000, 9800)


AT                \
                                                     Paddy rice         Wheat   
AT Paddy rice                                               0.0  0.000000e+00   
   Wheat                                                    0.0  0.000000e+00   
   Cereal grains nec                                        0.0  0.000000e+00   
   Vegetables, fruit, nuts                                  0.0  0.000000e+00   
   Oil seeds                                                0.0  0.000000e+00   
...                                                         ...           ...   
YE Membership organisation services n.e.c. (91)             0.0  2.201948e-20   
   Recreational, cultural and sporting services (92)        0.0  7.658744e-19   
   Other services (93)                                      0.0  4.373019e-20   
   Private households with employed persons (95)            0.0  2.343709e-22   
   Extra-territorial organizations and bodies               0.0  0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  0.000000e+00   
   Cereal grains nec                                      0.000000e+00   
   Vegetables, fruit, nuts                                0.000000e+00   
   Oil seeds                                              0.000000e+00   
...                                                                ...   
YE Membership organisation services n.e.c. (91)           1.160716e-20   
   Recreational, cultural and sporting services (92)      4.597331e-19   
   Other services (93)                                    2.369421e-20   
   Private households with employed persons (95)          1.325969e-22   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        0.000000e+00   
   Cereal grains nec                                            0.000000e+00   
   Vegetables, fruit, nuts                                      0.000000e+00   
   Oil seeds                                                    0.000000e+00   
...                                                                      ...   
YE Membership organisation services n.e.c. (91)                 5.443575e-20   
   Recreational, cultural and sporting services (92)            5.027412e-19   
   Other services (93)                                          5.080445e-20   
   Private households with employed persons (95)                1.107572e-22   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              0.000000e+00   
   Cereal grains nec                                  0.000000e+00   
   Vegetables, fruit, nuts                            0.000000e+00   
   Oil seeds                                          0.000000e+00   
...                                                            ...   
YE Membership organisation services n.e.c. (91)       5.031903e-20   
   Recreational, cultural and sporting services (92)  8.655573e-19   
   Other services (93)                                5.922199e-20   
   Private households with employed persons (95)      4.179066e-22   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                             \
             

In [83]:
# sum up all columns to get the total impact for each region-sector pair
pdf_total_w = pdf_w.sum()
pdf_total_w

AT  Paddy rice                                           0.000000e+00
    Wheat                                                8.371599e-11
    Cereal grains nec                                    1.880606e-11
    Vegetables, fruit, nuts                              4.764173e-12
    Oil seeds                                            9.244339e-11
                                                             ...     
WM  Membership organisation services n.e.c. (91)         8.024792e-13
    Recreational, cultural and sporting services (92)    1.188677e-12
    Other services (93)                                  1.284846e-12
    Private households with employed persons (95)        9.223753e-13
    Extra-territorial organizations and bodies           0.000000e+00
Length: 9800, dtype: float64

In [84]:
pd.DataFrame(pdf_total_w).to_csv("csv/pdf-eutrophication-marine-water.csv", index=True)